In [1]:
from dotenv import find_dotenv,load_dotenv
from langchain_openai import AzureChatOpenAI


load_dotenv(find_dotenv())

True

In [2]:
model = AzureChatOpenAI(deployment_name="", model_name="", temperature=0,
                                     model_version="",
                                     openai_api_version="",azure_endpoint = '' 
                                     )

In [3]:
from langchain_community.utilities import SQLDatabase

In [5]:
db = SQLDatabase.from_uri(database_uri="sqlite:///database.db")

In [8]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("select * from tourism limit 10"))

sqlite
['tourism']
[(0, 'Northern', 'Delhi', 'Delhi', 'India Gate', 'War Memorial', '1921', 0.5, 4.6, 0, 'Yes', None, 'Historical', 'Yes', 2.6, 'Evening'), (1, 'Northern', 'Delhi', 'Delhi', "Humayun's Tomb", 'Tomb', '1572', 2.0, 4.5, 30, 'Yes', None, 'Historical', 'Yes', 0.4, 'Afternoon'), (2, 'Northern', 'Delhi', 'Delhi', 'Akshardham Temple', 'Temple', '2005', 5.0, 4.6, 60, 'Yes', None, 'Religious', 'No', 0.4, 'Afternoon'), (3, 'Northern', 'Delhi', 'Delhi', 'Waste to Wonder Park', 'Theme Park', '2019', 2.0, 4.1, 50, 'Yes', 'Monday', 'Environmental', 'Yes', 0.27, 'Evening'), (4, 'Northern', 'Delhi', 'Delhi', 'Jantar Mantar', 'Observatory', '1724', 2.0, 4.2, 15, 'Yes', None, 'Scientific', 'Yes', 0.31, 'Morning'), (5, 'Northern', 'Delhi', 'Delhi', 'Chandni Chowk', 'Market', '1700', 3.0, 4.2, 0, 'Yes', 'Sunday', 'Market', 'Yes', 0.25, 'Afternoon'), (6, 'Northern', 'Delhi', 'Delhi', 'Lotus Temple', 'Temple', '1986', 1.0, 4.5, 0, 'Yes', 'Monday', 'Religious', 'Yes', 0.59, 'Evening'), (7, 'N

In [14]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(model,db)
response = chain.invoke({"question":"How many Cities are there?"})
response

'SELECT COUNT(DISTINCT "City") FROM tourism'

In [15]:
db.run(response.replace("\n",""))

'[(214,)]'

In [12]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

### Execute query using tool

In [13]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
query = create_sql_query_chain(model,db)
chain = query | execute_query
chain.invoke({"question":"How many Cities there?"})


'[(214,)]'

### Agent

In [23]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

agent_executor = create_sql_agent(model, db=db, toolkit=SQLDatabaseToolkit(db=db, llm=model), verbose=True)
agent_executor.invoke({
    "input" : "List Down the cities which are have rating higher than 4 and there significance are historical and religious"
})



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: tourism
Thought:The "tourism" table seems relevant. I should query its schema to see what columns are available.
Action: sql_db_schema
Action Input: tourism
Observation: 
CREATE TABLE tourism (
	"Unnamed: 0" INTEGER, 
	"Zone" TEXT, 
	"State" TEXT, 
	"City" TEXT, 
	"Name" TEXT, 
	"Type" TEXT, 
	"Establishment Year" TEXT, 
	"time needed to visit in hrs" REAL, 
	"Google review rating" REAL, 
	"Entrance Fee in INR" INTEGER, 
	"Airport with 50km Radius" TEXT, 
	"Weekly Off" TEXT, 
	"Significance" TEXT, 
	"DSLR Allowed" TEXT, 
	"Number of google review in lakhs" REAL, 
	"Best Time to visit" TEXT
)

/*
3 rows from tourism table:
Unnamed: 0	Zone	State	City	Name	Type	Establishment Year	time needed to visit in hrs	Google review rating	Entrance Fee in INR	Airport with 50km Radius	Weekly Off	Significance	DSLR Allowed	Number of google review in lakhs	Best Time to visit
0	Northern	Delhi	Delhi	India Gate	

{'input': 'List Down the cities which are have rating higher than 4 and there significance are historical and religious',
 'output': 'Delhi, Mumbai'}